In [ ]:
%matplotlib inline
import os, sys
import numpy as np
import geopandas as gpd
import pandas as pd
from plotnine.stats.stat_boxplot import weighted_percentile
import matplotlib.pyplot as plt
from onstove import OnStove, DataProcessor, RasterLayer, VectorLayer, Technology
from helper_functions import get_tariff, swatches, plot_baseline_split

In [ ]:
snake = True  # Change this to False to run the notebook by yourself and not through snakemake

# Setup
## Choose the scenario to run
Choose betwee `Social_optimum`, `High_sub_lpg`, `Low_sub_lpg`, and `Stated_policies`.

In [ ]:
if snake:
    scenario = snakemake.params.scenario
    gis_data_directory = snakemake.params.processed_data
    input_data_directory = os.path.join(snakemake.params.scenario_inputs, scenario)
    output_directory = os.path.join(snakemake.params.results_folder, scenario)
else:
    scenario = 'High_sub_lpg'
    gis_data_directory = os.path.join('..', '4. Results', 'Processed GIS Data')
    input_data_directory = os.path.join('..', '2. Scenario inputs', scenario)
    output_directory = os.path.join('..', '4. Results', scenario)
    
exchange_rate = 123.9838 # NPR/USD

## Go to the relevant step to run
1. [Model preparation](#Model-preparation)
2. [Model run](#Model-run)
3. [Plotting and saving maps](#Plotting-and-saving-maps)

# Model preparation

## 1. Create an OnSSTOVE model

In [ ]:
nepal = OnStove(project_crs=3857)
nepal.output_directory = output_directory

## 2. Read the model data

In [ ]:
path = os.path.join(input_data_directory, 'Nepal_soc.csv')
nepal.read_scenario_data(path, delimiter=',')

## 3. Add a country mask layer

In [ ]:
path = os.path.join(gis_data_directory, 'Administrative', 'Country_boundaries', 'Country_boundaries.geojson')
mask_layer = VectorLayer('admin', 'adm_0', path=path)
nepal.mask_layer = mask_layer

## 4. Add a population base layer

In [ ]:
path = os.path.join(gis_data_directory, 'Demographics', 'Population', 'Population.tif')
nepal.add_layer(category='Demographics', name='Population', path=path, layer_type='raster', base_layer=True)
nepal.population_to_dataframe()

## 5. Calibrate population and urban/rural split

In [ ]:
ghs_path = os.path.join(gis_data_directory, 'Demographics', 'Urban_rural_divide', 'Urban_rural_divide.tif')
nepal.calibrate_urban_rural_split(ghs_path)

## 6. Add wealth index GIS data

In [ ]:
wealth_path = os.path.join(gis_data_directory, 'Demographics', 'Wealth', 'Wealth.tif')
wealth_index = RasterLayer('Demographics', 'relative_wealth', path=wealth_path)
nepal.raster_to_dataframe(wealth_index, name='relative_wealth', method='read', 
                          fill_nodata_method='interpolate')

## 7. Calculate value of time

In [ ]:
# Based on wealth index, minimum wage and a lower and upper range for the cost of opportunity
nepal.get_value_of_time()

## 8. Read electricity network GIS layers

In [ ]:
# Read MV lines
path = os.path.join(gis_data_directory, 'Electricity', 'MV_lines', 'MV_lines.geojson')
mv_lines = VectorLayer('Electricity', 'MV_lines', path=path)

# Read MG
path = os.path.join(gis_data_directory, 'Electricity', 'MG_access', 'MG_access.geojson')
mg_access = VectorLayer('Electricity', 'MG_access', path=path)

### 8.1. Calculate distance to electricity infrastructure 

In [ ]:
nepal.distance_to_electricity(mv_lines=mv_lines)

### 8.2. Add night time lights data

In [ ]:
path = os.path.join(gis_data_directory, 'Electricity', 'Night_time_lights', 'Night_time_lights.tif')
ntl = RasterLayer('Electricity', 'Night_time_lights', path=path)

nepal.raster_to_dataframe(ntl, name='Night_lights', method='read')

## 9. Calibrate current electrified population

In [ ]:
nepal.current_elec()
nepal.final_elec()

print('Calibrated grid electrified population fraction:', nepal.gdf['Elec_pop_calib'].sum() / nepal.gdf['Calibrated_pop'].sum())

## 10. Read the cooking technologies data

In [ ]:
path = os.path.join(input_data_directory, 'Nepal_tech.csv')
nepal.read_tech_data(path, delimiter=',')

## 11. Adding Electric cooking data

### 11.1. Mini grids coverage and ecooking potential

In [ ]:
mg_dist_path = os.path.join(gis_data_directory, 'Electricity', 'MG_points', 'MG_points_dist.tif')
mg_access_path = os.path.join(gis_data_directory, 'Electricity', 'MG_access', 'MG_access.geojson')
ntl_path = os.path.join(gis_data_directory, 'Electricity', 'Night_time_lights', 'Night_time_lights.tif')

nepal.techs['Mini_Grids'].distance = mg_dist_path
nepal.techs['Mini_Grids'].coverage = mg_access_path
nepal.techs['Mini_Grids'].ntl = ntl_path

### 11.2. Adding percentage of households that need to upgrade connection and electricity cost 

In [ ]:
breaker_upgrade = VectorLayer(path=os.path.join('..', '1. Data', 'Breaker size & monthly elec cost GDF', 'province_breakersize.geojson'))

In [ ]:
if scenario in ['High_sub_lpg', 'Low_sub_lpg', 'Stated_policies']: 
    electricity_tariffs = pd.read_csv(os.path.join('..', '1. Data', 'Breaker size & monthly elec cost GDF', 'electricity_tariffs.csv'))
    breaker_upgrade.data.rename(columns={'5(Amps)': 5, '6-15(Amps)': 15, '16-30(Amps)': 30, '31-60(Amps)': 60}, inplace=True)
    breaker_melted = breaker_upgrade.data.melt(id_vars=['Province'],
                                               value_vars=[5, 15, 30, 60])
    breaker_melted.loc[breaker_melted['variable']==5, 'variable'] = 15

    nepal.techs['Electricity'].required_energy(nepal)
    breaker_melted['elec_cost'] = breaker_melted.apply(lambda row: get_tariff(row, 
                                                                              nepal.techs['Electricity'].energy,
                                                                              electricity_tariffs, 
                                                                              exchange_rate), 
                                                       axis=1)
    
    for province in breaker_melted['Province'].unique():
        dff = breaker_melted.loc[breaker_melted['Province'] == province, ['elec_cost', 'value']]
        cost = sum(dff['value'] * dff['elec_cost']) / dff['value'].sum()
        breaker_upgrade.data.loc[breaker_upgrade.data['Province']==province, 'elec_cost'] = cost    

In [ ]:
elec_data = nepal.gdf[['Households', 'geometry']].sjoin(breaker_upgrade.data[['%_households_upgrade', 'elec_cost', 'geometry']], how='left')

In [ ]:
if scenario in ['High_sub_lpg', 'Low_sub_lpg', 'Stated_policies']: 
    nepal.techs['Electricity'].fuel_cost = elec_data['elec_cost']
    nepal.techs['Mini_Grids'].fuel_cost = elec_data['elec_cost']
nepal.techs['Electricity'].connection_cost *= elec_data['%_households_upgrade']

### 11.3. Calculating grid added capacity cost

In [ ]:
nepal.techs['Electricity'].get_capacity_cost(nepal)

## 12. Reading GIS data for LPG supply

In [ ]:
lpg = RasterLayer('LPG', 'LPG_traveltime', 
                  os.path.join(gis_data_directory, 'LPG', 'LPG_traveltime', 'LPG_traveltime.tif'))
nepal.techs['LPG'].travel_time = nepal.raster_to_dataframe(lpg,
                                                           fill_nodata_method='interpolate', 
                                                           method='read') * 2 / 60

## 13. Adding GIS data for Traditional and Improved Biomass

In [ ]:
nepal.techs['Traditional_Biomass'].friction_path = os.path.join(gis_data_directory, 'Biomass', 'Friction', 'Friction.tif')
nepal.techs['Traditional_Biomass'].forest_path = os.path.join(gis_data_directory, 'Biomass', 'Forest', 'Forest.tif')
nepal.techs['Traditional_Biomass'].forest_condition = lambda x: x > 30

nepal.techs['Biomass_ICS_T2'].friction_path = os.path.join(gis_data_directory, 'Biomass', 'Friction', 'Friction.tif')
nepal.techs['Biomass_ICS_T2'].forest_path = os.path.join(gis_data_directory, 'Biomass', 'Forest', 'Forest.tif')
nepal.techs['Biomass_ICS_T2'].forest_condition = lambda x: x > 30

nepal.techs['Biomass_ICS_T3'].friction_path = os.path.join(gis_data_directory, 'Biomass', 'Friction', 'Friction.tif')
nepal.techs['Biomass_ICS_T3'].forest_path = os.path.join(gis_data_directory, 'Biomass', 'Forest', 'Forest.tif')
nepal.techs['Biomass_ICS_T3'].forest_condition = lambda x: x > 30

## 14. Adding GIS data for Biogas

In [ ]:
buffaloes = os.path.join(gis_data_directory, 'Biogas', 'Livestock', 'buffaloes', 'buffaloes.tif')
cattles = os.path.join(gis_data_directory, 'Biogas', 'Livestock', 'cattle', 'cattle.tif')
poultry =os.path.join(gis_data_directory, 'Biogas', 'Livestock', 'poultry', 'poultry.tif')
goats = os.path.join(gis_data_directory, 'Biogas', 'Livestock', 'goats', 'goats.tif')
pigs = os.path.join(gis_data_directory, 'Biogas', 'Livestock', 'pigs', 'pigs.tif')
sheeps = os.path.join(gis_data_directory, 'Biogas', 'Livestock', 'sheeps', 'sheeps.tif')

nepal.techs['Biogas'].temperature = os.path.join(gis_data_directory, 'Biogas', 'Temperature', 'Temperature.tif')
nepal.techs['Biogas'].recalibrate_livestock(nepal, buffaloes, cattles, poultry, goats, pigs, sheeps)
nepal.techs['Biogas'].friction_path = os.path.join(gis_data_directory, 'Biomass', 'Friction', 'Friction.tif')

## 15. Ploting the baseline technology split

In [ ]:
labels = {"Biogas and Electricity": "Electricity and Biogas",
          "Biogas and LPG": "LPG and Biogas",
          "Biogas and Biomass ICS T3": "Biomass ICS T3 and Biogas",
          "Biogas and Biomass ICS T2": "Biomass ICS T2 and Biogas",
          "Biogas and Mini Grids": "Mini Grids and Biogas"}

others = pd.DataFrame({'tech': ['Others', 'Others'], 'Population': [428384.8408, 462490.8129], 'region': ['Urban', 'Rural']})
plot_baseline_split(model=nepal, labels=labels, others=others)

## 16. Saving the prepared model inputs

In [ ]:
os.makedirs(nepal.output_directory, exist_ok=True)
nepal.to_pickle("model.pkl")

[Get back to setup](#Setup)

# Model run

## 1. Read the OnSSTOVE model

In [ ]:
nepal = OnStove.read_model(os.path.join(output_directory, 'model.pkl'))
nepal.output_directory = output_directory

## 2. Adding subsidies

### 2.1. Biomass ICS Tier 3 stoves
According to the [Renewable Energy Subsidy Policy 2022](https://www.aepc.gov.np/uploads/docs/uu-a-1648949209.pdf) subsidies will be given to:
>50% subsidy for one or two pot hole with full or partial metal body rocket (not to exceed Rs. 3,000) & gasifier stoves (not to exceed Rs. 4,000) in rural and peri-urban areas

In [ ]:
if scenario in ['Stated_policies', 'High_sub_lpg', 'Low_sub_lpg']:
    nepal.techs['Biomass_ICS_T3'].inv_cost = pd.Series(nepal.techs['Biomass_ICS_T3'].inv_cost * np.ones(nepal.gdf.shape[0]), 
                                                       index=nepal.gdf.index)
    nepal.techs['Biomass_ICS_T3'].inv_cost.loc[(nepal.gdf['IsUrban'] < 20)] *= 0.5

### 2.2. Biogas digester investments
According to the [Renewable Energy Subsidy Policy 2022](https://www.aepc.gov.np/uploads/docs/uu-a-1648949209.pdf), subsidies will be provided for the investments in biodigesters depending on their size and ecological zone:
Region|2 cubic meters|4 cubic meters|6 cubic meters and above|Average (Rs)|Average (USD)
:---|---:|---:|---:|---:|---:
Mountain districts|32,500|39,000|45,500|39,000|314
Hill districts|26,000|32,500|39,000|32,500|262
Terai districts|19,200|24,000|28,800|24,000|193

We use the average subsidie per ecological zone.

In [ ]:
if scenario in ['High_sub_lpg', 'Stated_policies', 'Low_sub_lpg']:
    eco_zones = VectorLayer(path=os.path.join('..', '1. Data', 'Ecological zones', 'Ecological zones.geojson'))
    eco_zones.reproject(3857)
    zones = nepal.gdf[['Households', 'geometry']].sjoin(eco_zones.data[['zone', 'geometry']], how='left')
    nepal.gdf['eco_zone'] = zones['zone']
    nepal.techs['Biogas'].inv_cost = pd.Series(nepal.techs['Biogas'].inv_cost * np.ones(nepal.gdf.shape[0]), 
                                               index=nepal.gdf.index)
    nepal.techs['Biogas'].inv_cost.loc[(nepal.gdf['eco_zone']=='Terai')] -= 193.5736766
    nepal.techs['Biogas'].inv_cost.loc[(nepal.gdf['eco_zone']=='Hills')] -= 262.1310203
    nepal.techs['Biogas'].inv_cost.loc[(nepal.gdf['eco_zone']=='Mountain')] -= 314.5572244

### 2.3. Moving fuel subsidies from LPG to Electricity
Currently the Nepal Oil Corporation says it buys LPG at Rs2,551.15 per cylinder and sells it for Rs1,800 incurring on a loss of Rs751.14 on every cylinder sold: https://kathmandupost.com/national/2022/06/07/oil-utility-mulls-reviving-dual-pricing-on-cooking-gas-insiders-sceptical 

We take the sell price and reduce the subsidy while we increase the subsidy for Electricity, until the same subsidy is reached in terms of cost/energy unit. 

In [ ]:
if scenario in ['Stated_policies']:
    increment_percentage = 1 # in percentage of subsidy
    increment = 549.875 * increment_percentage / (exchange_rate * 14.2) # USD per kg
    nepal.techs['LPG'].fuel_cost += increment
    
    increment_per_MJ = increment / nepal.techs['LPG'].energy_content # in USD/MJ
    decrease_per_kwh = increment_per_MJ * nepal.techs['Electricity'].energy_content # in USD/kWh

    el_sub_factor = 0.5
    nepal.techs['Electricity'].fuel_cost -= decrease_per_kwh * el_sub_factor
    nepal.techs['Mini_Grids'].fuel_cost -= decrease_per_kwh * el_sub_factor

## 3. Calculating benefits and costs of each technology and getting the max benefit technology for each cell

In [ ]:
names = ['Electricity', 'Mini_Grids', 'LPG', 'Biogas', 'Traditional_Biomass', 'Biomass_ICS_T2', 'Biomass_ICS_T3']
nepal.run(technologies=names)

## 4. Printing the results

In [ ]:
nepal.summary()

## 5. Saving the results

In [ ]:
nepal.to_pickle("results.pkl")
nepal.summary().to_csv(os.path.join(nepal.output_directory, 'summary.csv'), index=False)

[Get back to setup](#Setup)

# Plotting and saving maps

## 1. Reading the results and creating the colormap

In [ ]:
nepal = OnStove.read_model(os.path.join(output_directory, 'results.pkl'))

In [ ]:
cmap = {"Biomass ICS T2": '#B6195E',
        "Biomass ICS T2 and Biogas": "#F6029E", 
        "Biomass ICS T3": '#57365A', 
        "Biomass ICS T3 and Biogas": "#bd57b6", 
        "Biogas": '#73af48', 
        "LPG": '#40bdbc', 
        "LPG and Biogas": "#156e5e", 
        "Electricity": '#d14a36',
        "Electricity and Biomass ICS T2": "#FF0000",
        "Electricity and Biomass ICS T3": "#e5a3a3", 
        "Electricity and Biogas": "#e17c05",
        "Electricity and LPG": "#FFC107", 
        "Electricity and Mini Grids": "#ffd9df",
        "Mini Grids": "#0096FF", #ff7678
        "Mini Grids and Biogas": "#0047AB",
        "Traditional Biomass": 'black'} #ffd9df

labels = {"Biogas and Electricity": "Electricity and Biogas",
          "Biogas and LPG": "LPG and Biogas",
          "Biogas and Biomass ICS T3": "Biomass ICS T3 and Biogas",
          "Biogas and Biomass ICS T2": "Biomass ICS T2 and Biogas",
          "Biogas and Mini Grids": "Mini Grids and Biogas"}

scale_bar_prop = dict(size=100000, style='double', textprops=dict(size=11), location=(0.5, 0.02),
                      linekw=dict(lw=1, color='black'), extent=0.01, loc='lower center')
north_arrow_prop = dict(size=30, location=(0.92, 0.92), linewidth=0.5)

In [ ]:
# Check how the colormap looks by calling the swatches helper function
swatches(cmap)

## 2. Calculating subsidies

In [ ]:
## LPG
subsidy_lpg = 0
if scenario != 'Stated_policies':
    subsidy_hh_lpg = nepal.techs['LPG'].energy / nepal.techs['LPG'].energy_content / 14.2 * 549.875 / exchange_rate
    subsidy_lpg = nepal.gdf.loc[nepal.gdf['max_benefit_tech'] == 'LPG', 'Households'].sum() * subsidy_hh_lpg / 1000000000

## Biomass ICS
subsidy_ics = (nepal.gdf.loc[nepal.gdf['max_benefit_tech']=='Biomass_ICS_T3', 'Households'] * nepal.techs['Biomass_ICS_T3'].inv_cost).sum() / 1000000000
if scenario == 'Social_optimum':
    subsidy_ics *= 0.5

## Biogas
eco_zones = VectorLayer(path=os.path.join('..', '1. Data', 'Ecological zones', 'Ecological zones.geojson'))
eco_zones.reproject(3857)
zones = nepal.gdf[['Households', 'geometry']].sjoin(eco_zones.data[['zone', 'geometry']], how='left')
nepal.gdf['eco_zone'] = zones['zone']
subsidy_biogas = nepal.gdf.loc[(nepal.gdf['max_benefit_tech'] == 'Biogas') & 
                               (nepal.gdf['eco_zone'] == 'Terai'), 'Households'].sum() * 193.5736766
subsidy_biogas += nepal.gdf.loc[(nepal.gdf['max_benefit_tech'] == 'Biogas') & 
                                (nepal.gdf['eco_zone'] == 'Hills'), 'Households'].sum() * 262.1310203
subsidy_biogas += nepal.gdf.loc[(nepal.gdf['max_benefit_tech'] == 'Biogas') & 
                                (nepal.gdf['eco_zone'] == 'Mountain'), 'Households'].sum() * 314.5572244
subsidy_biogas /= 1000000000

## Electricity
subsidy_electricity = 0
if scenario in ['Stated_policies']:
    increment = 549.875 / (exchange_rate * 14.2) # USD per kg
    
    increment_per_MJ = increment / nepal.techs['LPG'].energy_content # in USD/MJ
    subsidy_per_kwh = increment_per_MJ * nepal.techs['Electricity'].energy_content # in USD/kWh
    
    kwh_grid = nepal.techs['Electricity'].energy / nepal.techs['Electricity'].energy_content
    kwh_minigrid = nepal.techs['Mini_Grids'].energy / nepal.techs['Mini_Grids'].energy_content
    
    subsidy_electricity = nepal.gdf.loc[nepal.gdf['max_benefit_tech'] == 'Electricity', 'Households'].sum() * kwh_minigrid * subsidy_per_kwh * el_sub_factor
    subsidy_electricity += nepal.gdf.loc[nepal.gdf['max_benefit_tech'] == 'Mini_Grids', 'Households'].sum() * kwh_minigrid * subsidy_per_kwh * el_sub_factor
    subsidy_electricity /= 1000000000
    
all_subsidies = subsidy_lpg + subsidy_ics + subsidy_biogas + subsidy_electricity

# Calculating total system cost
summary = nepal.summary(total=True, pretty=False, variable='max_benefit_tech', remove_none=True)
total_costs = (summary.loc['total', 'investment_costs'] + summary.loc['total', 'fuel_costs'] + 
               summary.loc['total', 'om_costs'] - summary.loc['total', 'salvage_value']) / 1000

if scenario != 'Social_optimum':
    total_costs += all_subsidies

## 3. Plotting the results

In [ ]:
subsidies_df = pd.DataFrame({'LPG subsidies (BUS$)': [subsidy_lpg],
                             'Biomass ICS T3 subsidies (BUS$)': [subsidy_ics],
                             'Biogas subsidies (BUS$)': [subsidy_biogas],
                             'Electricity subsidies (BUS$)': [subsidy_electricity]},
                            index=['Subsidies prioritized'])
subsidies_df.to_csv(os.path.join(nepal.output_directory, f'Subsidies_{scenario}.csv'))
subsidies_df

### 3.1. Plot optimal technology mix map

In [ ]:
nepal.plot('max_benefit_tech', cmap=cmap, legend_position=(0.62, 0.95), 
           figsize=(13, 9), title=f'Maximum benefit cooking technology | Nepal',  
           labels=labels, legend=True, legend_title='Maximum benefit\ncooking technology', 
           rasterized=True, stats=True,
           stats_kwargs={'extra_stats': {'Total system cost': f'{total_costs:.2f} BUS$',
                                         'Subsidies': f'{all_subsidies:.2f} BUS$'}, 
                         'fontsize': 14, 'stats_position': (0.05, 0.28), 'pad': 2, 'sep': 0},
           scale_bar=scale_bar_prop, north_arrow=north_arrow_prop,
           dpi=300, 
          )

In [ ]:
nepal.to_image('max_benefit_tech', cmap=cmap,
               figsize=(13, 9), dpi=300, 
               labels=labels, legend=False,
               rasterized=True, stats=True, name='max_benefit_tech.pdf',
               stats_kwargs={'extra_stats': {'Total system cost': f'{total_costs:.2f} BUS$',
                                         'Subsidies': f'{all_subsidies:.2f} BUS$'}, 
                             'fontsize': 14, 'stats_position': (0, 0.28), 'pad': 2, 'sep': 0},
               scale_bar=scale_bar_prop, north_arrow=north_arrow_prop)

### 3.2. Plot technology split

In [ ]:
nepal.plot_split(cmap=cmap, labels=labels, height=2, width=3.5, orientation='h',
                 labs_kwargs=dict(x=''), save_as='tech_split.pdf')

### 3.3. Plot costs and benefits

In [ ]:
nepal.plot_costs_benefits(labels=labels, height=2, width=5.5, save_as='costs_benefits.pdf')

### 3.4. Plot optimal technology distribution over relative wealth

In [ ]:
plt.clf()

# Draw histogram
max_val = nepal.gdf['relative_wealth'].max()
min_val = nepal.gdf['relative_wealth'].min()
binwidth = (max_val - min_val) * 0.05

fig = nepal.plot_distribution(type='histogram', variable='relative_wealth', groupby='None', cmap=cmap, labels=labels, 
                              hh_divider=1000, y_title='Households (k)', font_args=dict(color='black', size=11),
                              height=2.5, width=4, kwargs=dict(binwidth=binwidth, alpha=0.8, size=0.2), quantiles=True)

# Add quantile lines
q1, q3 = weighted_percentile(a=nepal.gdf['relative_wealth'].values, q=(25, 75), weights=nepal.gdf['Households'].values)

# get axes to annotate
ax = fig.axes[0] # get the matplotlib axis (may be more than one if faceted)

# annotate percentages
dff = nepal.gdf.groupby('max_benefit_tech')['Households'].sum()
dff.sort_values(inplace=True)
total = dff.sum()

## Dictionaries to set the possition of the arrows and text in the histogram
y_arrow_social_private = {'Electricity': 400, 'LPG': 180, 'Biogas': 50, 'Biomass_ICS_T3': 200, 'Mini_Grids': 0}
y_arrow_lpg_high = {'Electricity': 100, 'LPG': 400, 'Biogas': 50, 'Biomass_ICS_T3': 200, 'Mini_Grids': 0}
y_arrow_subsidies = {'Electricity': 400, 'LPG': 180, 'Biogas': 80, 'Biomass_ICS_T3': 50, 'Mini_Grids': 0}
y_arrow_dict = {'Social_optimum': y_arrow_social_private, 'High_sub_lpg': y_arrow_lpg_high,
                'Low_sub_lpg': y_arrow_social_private, 'Stated_policies': y_arrow_subsidies}
y_text_subsidies = {'Electricity': 500, 'LPG': 300, 'Biogas': 400, 'Biomass_ICS_T3': 250, 'Mini_Grids': 100}
y_text_general = {'Electricity': 500, 'LPG': 500, 'Biogas': 250, 'Biomass_ICS_T3': 400, 'Mini_Grids': 100}
y_text_lpg_high = {'Electricity': 500, 'LPG': 550, 'Biogas': 250, 'Biomass_ICS_T3': 400, 'Mini_Grids': 100}
y_text_dict = {'Social_optimum': y_text_general, 'High_sub_lpg': y_text_lpg_high,
               'Low_sub_lpg': y_text_general, 'Stated_policies': y_text_subsidies}
x_text_subsidies = {'Electricity': 1.5, 'LPG': 2, 'Biogas': -0.9, 'Biomass_ICS_T3': -1, 'Mini_Grids': -1.3}
x_text_general = {'Electricity': 1.5, 'LPG': -1.1, 'Biogas': -1.1, 'Biomass_ICS_T3': -1, 'Mini_Grids': -1.3}
x_text_lpg_high = {'Electricity': 1.8, 'LPG': -0.8, 'Biogas': -1.2, 'Biomass_ICS_T3': -1, 'Mini_Grids': -1.3}
x_text_dict = {'Social_optimum': x_text_general, 'High_sub_lpg': x_text_lpg_high,
               'Low_sub_lpg': x_text_general, 'Stated_policies': x_text_subsidies}
## Draw arrows and text
for tech in dff.index:
    group = nepal.gdf.loc[nepal.gdf['max_benefit_tech']==tech]
    hh = group['Households'].sum()
    share = hh/total
    if share > 0.01:
        x = weighted_percentile(a=group['relative_wealth'].values, q=(50), weights=group['Households'].values)
        y_arrow = y_arrow_dict[scenario][tech]
        y_text = y_text_dict[scenario][tech]
        x_text = x_text_dict[scenario][tech]

        ax.annotate(xy=(x, y_arrow),
                    xytext=(x_text, y_text), text=f'{share:.0%}',
                    horizontalalignment='right', verticalalignment='center',
                    arrowprops={'arrowstyle': '-', 'color': '#4D4D4D', 'lw': 0.5},
                    color='#4D4D4D')
        
fig.set_size_inches(4, 2.5)
fig.savefig(os.path.join(nepal.output_directory, 'tech_histogram.pdf'), bbox_inches='tight', transparent=True)
fig

## 4. Saving the results as raster files

In [ ]:
nepal.gdf['total_annualized_costs'] = nepal.gdf['investment_costs'] + nepal.gdf['fuel_costs'] + nepal.gdf['om_costs'] - nepal.gdf['salvage_value']

In [ ]:
nepal.to_raster('max_benefit_tech', cmap=cmap, labels=labels, nodata=0, mask=True, mask_nodata=65535)
nepal.to_raster('maximum_net_benefit', metric='per_household', nodata=0, mask=True, mask_nodata=-999999)
nepal.to_raster('maximum_net_benefit', metric='total', nodata=0, mask=True, mask_nodata=-999999)
nepal.to_raster('net_benefit_LPG', metric='per_household')
nepal.to_raster('net_benefit_Biogas', metric='per_household')
nepal.to_raster('net_benefit_Biomass_ICS_T3', metric='per_household')
nepal.to_raster('net_benefit_Electricity', metric='per_household')
nepal.to_raster('net_benefit_Mini_Grids', metric='per_household')
nepal.to_raster('total_annualized_costs', metric='per_household', nodata=0, mask=True, mask_nodata=-999999)
nepal.to_raster('total_annualized_costs', metric='total', nodata=0, mask=True, mask_nodata=-999999)
nepal.to_raster('investment_costs', metric='per_household', nodata=0, mask=True, mask_nodata=-999999)
nepal.to_raster('investment_costs', metric='total', nodata=0, mask=True, mask_nodata=-999999)
nepal.to_raster('fuel_costs', metric='per_household', nodata=0, mask=True, mask_nodata=-999999)
nepal.to_raster('fuel_costs', metric='total', nodata=0, mask=True, mask_nodata=-999999)
nepal.to_raster('deaths_avoided', metric='per_100k', nodata=0, mask=True, mask_nodata=-999999)
nepal.to_raster('time_saved', metric='per_household')
nepal.to_raster('reduced_emissions', metric='per_household', nodata=0, mask=True, mask_nodata=-999999)

nepal.to_raster('reduced_emissions', metric='total', nodata=0, mask=True, mask_nodata=-999999)
raster = RasterLayer(path=f'../4. Results/{scenario}/Rasters/reduced_emissions_total.tif')
raster.data[raster.data!=raster.meta['nodata']] /= 1000
raster.name = 'reduced_emissions_total'
raster.save(f'../4. Results/{scenario}/Rasters')

nepal.to_raster('health_costs_avoided', metric='per_household', nodata=0, mask=True, mask_nodata=-999999)
nepal.to_raster('health_costs_avoided', metric='total', nodata=0, mask=True, mask_nodata=-999999)
nepal.to_raster('Households', metric='sum', nodata=0, mask=True, mask_nodata=-999999)
nepal.to_raster('available_biogas', metric='mean', nodata=0, mask=True, mask_nodata=-999999)

[Get back to setup](#Setup)